# YOLO v9を試す
今回の課題は物体検出タスクである。  
渋谷交差点を定点カメラから撮影した画像を1分ごとに撮影したものだが、実際に用いる際はリアルタイム性が必要だと判断しYOLO v9をベースとして用いる。

In [4]:
from ultralytics import YOLO

model = YOLO("yolov9c.pt")

100%|██████████| 49.4M/49.4M [00:12<00:00, 4.22MB/s]


In [14]:
for h in range(0, 24, 1):
    for m in range(0, 60, 1):
        results = model(f"../data/20240512/20240512_{str(h).zfill(2)}{str(m).zfill(2)}00.jpg", save=True, save_txt=True, save_conf=True, show=True)



image 1/1 F:\src\JPT\shibuya-scramble\notebooks\..\data\20240512\20240512_000000.jpg: 384x640 18 persons, 3 cars, 1 tv, 713.1ms
Speed: 1.0ms preprocess, 713.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to F:\src\JPT\shibuya-scramble\runs\detect\predict
60 labels saved to F:\src\JPT\shibuya-scramble\runs\detect\predict\labels

image 1/1 F:\src\JPT\shibuya-scramble\notebooks\..\data\20240512\20240512_000100.jpg: 384x640 6 persons, 8 cars, 3 traffic lights, 1 tv, 690.3ms
Speed: 2.0ms preprocess, 690.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to F:\src\JPT\shibuya-scramble\runs\detect\predict
60 labels saved to F:\src\JPT\shibuya-scramble\runs\detect\predict\labels

image 1/1 F:\src\JPT\shibuya-scramble\notebooks\..\data\20240512\20240512_000200.jpg: 384x640 8 persons, 10 cars, 1 bus, 1 traffic light, 1 tv, 688.3ms
Speed: 1.0ms preprocess, 688.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


20240512に存在する画像から物体検出を行った。
/runs に生成された画像を確認すると、正誤はともかく `cars`/`trucks`/`bus`/`motorcycles` などが検出できている。  
このままで顧客の要件を満たすかわからないため、目視で確認した範囲で見つけた　`乗用車`/`タクシー`/`バス`/`ゴミ収集車`/`トラック`/`救急車`/`パトカー`/`原付` を分類する方が望ましいか顧客に確認を行う。(依頼: 2024/06/03)
